<h1>Running Regression Models for MLP + LSTM</h1>

In [1]:
import os
import time

In [2]:
os.getcwd()

'C:\\Users\\13473\\Desktop\\e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664\\Ipynbs To Run'

In [3]:
wd_path = r'../'
os.chdir(wd_path)
wd_path=os.getcwd()

In [4]:
import pandas as pd
import numpy as np

from datetime import datetime

In [5]:
normalized_data = 'complete_universe_cleaned_normalized_jonn.csv'
auto_encoder_data = "Autoencoder_transformed_43_dim_data.csv"

In [6]:
start_time = time.time()
df = pd.read_csv(normalized_data ,dtype={'COMNAM':str,'NCUSIP':str,'PERMNO':str,'PERMCO':str,'SICCD':str,'TICKER_PERMNO':str},low_memory=False,index_col=0)
df['DATE'] = pd.to_datetime(df['DATE'],format='%Y-%m-%d')
df = df.reset_index()
print("--- %s seconds ---" % (time.time() - start_time))

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


--- 45.088263750076294 seconds ---


In [ ]:
start_time = time.time()
auto = pd.read_csv(auto_encoder_data,low_memory=False,index_col=0)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df.columns

In [ ]:
identifier_cols = ['DATE','COMNAM','NCUSIP','PERMNO','PERMCO','SICCD','TICKER_PERMNO']
cross_sectional_cols=['accru', 'adv_sale', 'aftret_eq', 'aftret_equity','aftret_invcapx', 'at_tu', 'bm', 'CAPEI', 'capital_rat', 'cash_debt',
       'cash_lt', 'cash_rat', 'cfm', 'curr_debt', 'curr_rat', 'de_rat','debt_assets', 'debt_at', 'debt_capit', 'debt_ebitd', 'debt_invcap',
       'divyield', 'dltt_be', 'equity_invcap', 'evm', 'gpm', 'GProf', 'intcov','intcov_rat', 'invt_act', 'lt_debt', 'lt_ppent', 'npm', 'ocf_lct',
       'opmad', 'opmbd', 'pay_tu', 'pcf', 'pe_ex', 'pe_inc', 'pretret_earnat','pretret', 'profit_lct', 'ps', 'ptb', 'ptpm', 'quick_rat', 'rd_sale',
       'rect_act', 'rect_tu', 'roa', 'roce', 'roe', 'sale_equity','sale_invcap', 'short_debt', 'staff_sale', 'totdebt_invcap']
sector_cols = ['Agriculture, Forestry and Fishing', 'Construction','Finance, Insurance', 'Manufacturing', 'Mining', 'Nonclassifiable',
       'Public Administration', 'Real Estate', 'Retail Trade', 'Services','Transportation, Communications, Electric, Gas and Sanitary service',
       'Wholesale Trade', 'not used']
time_series_cols= ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1']
Y = ['m+1']
returns =  ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1','m+1']

In [ ]:
new_df = pd.concat([df[identifier_cols],auto,df[returns]], axis=1, sort=False)

In [ ]:
del df
del auto 
df = None
auto = None

In [ ]:
new_df.columns

In [ ]:
cross_sectional_cols = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42']
time_series_cols = ['m-12', 'm-11', 'm-10',
       'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2', 'm-1']
Y = ['m+1']

In [ ]:
len(cross_sectional_cols)

In [ ]:
len(time_series_cols)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

In [ ]:
'''Model 4: LSTM + MLP Regression'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(43,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(1, activation='sigmoid')(h3)

hybrid_model2=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

opt=tf.keras.optimizers.Adam(
    learning_rate=0.005,
)
hybrid_model2.compile(
    optimizer=opt, loss='binary_crossentropy',metrics=['binary_crossentropy','mean_squared_error','mean_absolute_error','mean_absolute_percentage_error']
)

In [ ]:
hybrid_model2.summary()

In [ ]:
dates = new_df['DATE'].unique()

In [ ]:
dates

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
import gc

In [ ]:
start_time = time.time()

x = [new_df[cross_sectional_cols].values,new_df[time_series_cols].values]
y = new_df[Y].values[:,0]

for dt in dates[119:]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    print(start_date,end_date)
    
    mask_train=(new_df[['DATE']]>=start_date) & (new_df[['DATE']]<=end_date)
    
    x_train, y_train=[i[mask_train['DATE'],:] for i in x], y[mask_train['DATE']]
    
    hybrid_model2=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

    opt=tf.keras.optimizers.Adam(
        learning_rate=0.005,
    )
    hybrid_model2.compile(
        optimizer=opt, loss='binary_crossentropy')
    
    mcp_save = ModelCheckpoint(os.path.join(wd_path,'hdf5_files\hybrid_reg_model_w_auto_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'), save_best_only=True, monitor='val_loss', mode='min') 
    
    history_hybrid2=hybrid_model2.fit(
        x=x_train, y=y_train, batch_size=512, epochs=3, verbose=1, 
        callbacks=[mcp_save],
        validation_split=0.1
    )
    
    del hybrid_model2
    del history_hybrid2
    del x_train
    del y_train
    gc.collect()
    hybrid_model2 = None
    history_hybrid2 = None
    x_train = None
    y_train = None
print("--- %s seconds ---" % (time.time() - start_time))